In [1]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import DESCNUpliftModelFactory

/home/ogrobertino/test_env/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Failed to import duecredit due to No module named 'duecredit'


In [3]:
DESCNUpliftModelFactory

src.factory.DESCNUpliftModelFactory

In [4]:
from src.models.NNUpliftModeling.DESCNUpliftModel import DESCNUpliftModel

In [5]:
ds_name = 'lazada_v2'
features_percent = 100
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)

In [6]:
import pandas as pd
ds = pd.read_csv(train_path, sep='\t')

In [7]:
input_size = len(ds.columns) - 2
input_size

83

In [58]:
params = {
    'input_dim': input_size,             # Должно быть задано в соответствии с данными
    'share_dim': [32], # Варианты размерности общих слоев
    'base_dim': [32],   # Варианты размерности базовых слоев
    'do_rate': [0.1, 0.2, 0.3],  # Варианты dropout
    'batch_norm': True, # Использование BatchNorm
    'normalization': 'none', # Тип нормализации
    'factual_loss_weight': [0.8, 1.0, 1.2], # Вес фактической потери
    'propensity_loss_weight': [0.05, 0.1, 0.2], # Вес потери пропенсити
    'tau_loss_weight': [0.05, 0.1, 0.2],    # Вес потери tau (если применимо)
    'batch_size': 32,
    'gradient_accumulation_steps' : 1,
    'epochs': 10
}

In [46]:
# params = {
#     'input_dim': input_size,             # Должно быть задано в соответствии с данными
#     'share_dim': [2], # Варианты размерности общих слоев
#     'base_dim': [2],   # Варианты размерности базовых слоев
#     'do_rate': [0.1, 0.2, 0.3],  # Варианты dropout
#     'batch_norm': [True, False], # Использование BatchNorm
#     'normalization': ['none', 'divide'], # Тип нормализации
#     'factual_loss_weight': [0.8, 1.0, 1.2], # Вес фактической потери
#     'propensity_loss_weight': [0.05, 0.1, 0.2], # Вес потери пропенсити
#     'tau_loss_weight': [0.05, 0.1, 0.2],    # Вес потери tau (если применимо)
#     'batch_size': 32,
#     'gradient_accumulation_steps' : 2,
#     'epochs': 1
# }

In [59]:
configs = DESCNUpliftModel.generate_config(count=4, params = params)

In [60]:
factory = DESCNUpliftModelFactory
config = configs[0]
batch_size=32
max_size=100000

In [61]:
model, train, test = factory.create(config, train_path, test_path)

In [12]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     train, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [66]:
kek = model.predict(train)

In [70]:
pd.DataFrame(kek['uplift'])

,0
0,-36.352661
1,-42.133018
2,-36.518047
3,-23.922583
4,-11.756742
...,...
694971,-34.046310
694972,-49.142448
694973,-48.341515
694974,-44.975857


In [88]:
q = model.predict(train)

In [92]:
q['df'][q['df']['treatment'] == 1]

,score,treatment,target
22,-35.111008,1.0,0.0
86,-2.519458,1.0,1.0
106,-2.163131,1.0,1.0
152,-13.546045,1.0,0.0
229,-2.354023,1.0,1.0
...,...,...,...
694711,-6.724401,1.0,1.0
694821,-2.642574,1.0,1.0
694829,-8.731159,1.0,1.0
694831,-5.754957,1.0,1.0


In [83]:
a[:, 0]

array([-36.35266 , -42.13302 , -36.518047, ..., -48.341515, -44.975857,
       -15.999656], dtype=float32)

In [85]:
pd.DataFrame({'a': a[:, 0]})

,a
0,-36.352661
1,-42.133018
2,-36.518047
3,-23.922583
4,-11.756742
...,...
694971,-34.046310
694972,-49.142448
694973,-48.341515
694974,-44.975857


In [96]:
a = q['df']

In [101]:
# a.to_csv()

In [105]:
train_test_model(ds_name='lazada_v2', features_percent=100, factory=DESCNUpliftModelFactory, config=configs[0], NN=True)   

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Модель, предсказания и конфиг сохранены в директории exps2/lazada_v2/100/6
Эксперимент сохранен в таблице /home/ogrobertino/UpliftModelingResearch/exps2/stats.tsv


In [ ]:
# for i in tqdm(range(len(configs)), desc="Training models"):
#     train_test_model(ds_name='lazada', features_percent=100, factory=DRModelFactory, config=configs[i])   